# Score : Didn't even test not good enough

#### The mappings can be better by quantifying by the median house price

In [1]:
import pandas as pd

master_training_data = pd.read_csv('../data/house_prices/train.csv')
master_testing_data = pd.read_csv('../data/house_prices/test.csv')

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer
from sklearn.svm import SVC
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
master_training_data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

### Removing shady columns.  Perhaps more to remove in the future

In [3]:
# MiscVal: $Value of miscellaneous feature
# MiscFeature: Miscellaneous feature not covered in other categories
# Utilities: Type of utilities available <-- removed because it's homogeneous
# Condition2: Proximity to main road or railroad (if a second is present) <-- removed because it's homogeneous
# PoolQC <--- Too little data
# 3SsnPorch: Three season porch area in square feet <-- removed because it's homogeneous
# Heating: Type of heating <-- removed because it's homogeneous
# Street: Type of road access <-- removed because it's homogeneous
# RoofMatl: Roof material <-- removed because it's homogeneous

# removed_cols = ['MiscVal', 'MiscFeature', 'BsmtQual', 'Utilities', "Condition2", "PoolQC", "Street", "3SsnPorch",
#                "Electrical", "Heating"]

# training_data = master_training_data.drop(removed_cols, axis=1)
# testing_data = master_testing_data.drop(removed_cols, axis=1)

training_data = master_training_data
testing_data = master_testing_data

#### Shortcut to modify both at the same time

In [4]:
full_data = [training_data, testing_data]

## Aggregating Quality and Condition

In [5]:
training_data['Quality'] = 0
testing_data['Quality'] = 0

qual_cols = ['BsmtQual', 'GarageQual', 'ExterQual', 'ExterCond', 'FireplaceQu', 'KitchenQual', 'BsmtCond', 'PoolQC',
            'HeatingQC']
qual_map = {"Po": 0, "Fa": 1, "TA": 2, "Gd": 3, "Ex": 4}

for col in qual_cols:
    training_data[col].fillna("TA", inplace=True)
    testing_data[col].fillna("TA", inplace=True)

for dataset in full_data:
    for col in qual_cols:
        dataset['Quality'] += dataset[col].map(qual_map)

training_data.drop(qual_cols, axis=1)
testing_data.drop(qual_cols, axis=1)
training_data["Quality"].head(5)

0    23
1    21
2    23
3    22
4    23
Name: Quality, dtype: int64

#### Understanding (and modifying) SaleCondition

In [6]:
sale_cond_map = {"Normal": 0, "Partial": 1, "Abnorml": 2, "Family": 3, "Alloca": 3, "AdjLand": 3}
for dataset in full_data:
    dataset['SaleCondition'] = dataset['SaleCondition'].map(sale_cond_map)

#### Sale Type

In [7]:
sale_type_map = {"WD": 0, "New": 1, "COD": 2, "ConLD": 2, "ConLw": 2, "ConLI": 2, "CWD": 2, "Oth": 2,"Con": 2}

for dataset in full_data:
    dataset['SaleType'] = dataset['SaleType'].map(sale_type_map)

#### Making the numbers in year sold smaller (does this do anything, I just feel compelled to do it)

In [8]:
for dataset in full_data:
    dataset['YrSold'] = dataset['YrSold'] - 2006

#### MSZoning: The general zoning classification

In [9]:
ms_zone_map = {"RL": 0, "RM": 1, "FV": 2, "RH": 2, "C (all)": 2}

for dataset in full_data:
    dataset['MSZoning'] = dataset['MSZoning'].map(ms_zone_map)

#### Alley: Type of alley access

In [10]:
ms_zone_map = {"Grvl": 0, "Pave": 2}

for dataset in full_data:
    dataset['Alley'] = dataset['Alley'].map(ms_zone_map)

In [11]:
training_data["Alley"].fillna(1, inplace=True)
testing_data["Alley"].fillna(1, inplace=True)

#### LotShape

In [12]:
lot_shape_map = {"Reg": 0, "IR1": 1, "IR2": 2, "IR3": 2}

for dataset in full_data:
    dataset['LotShape'] = dataset['LotShape'].map(lot_shape_map)

#### LandContour

In [13]:
lot_contour_map = {"Lvl": 0, "Bnk": 1, "HLS": 2, "Low": 3}

for dataset in full_data:
    dataset['LandContour'] = dataset['LandContour'].map(lot_contour_map)

#### LotConfig: Lot configuration

In [14]:
lot_config_map = {"Inside": 0, "Corner": 1, "CulDSac": 2, "FR2": 3, "FR3": 3}

for dataset in full_data:
    dataset['LotConfig'] = dataset['LotConfig'].map(lot_config_map)

#### LandSlope: Slope of property

In [15]:
land_slope_map = {"Gtl": 0, "Mod": 1, "Sev": 2}

for dataset in full_data:
    dataset['LandSlope'] = dataset['LandSlope'].map(land_slope_map)

#### Neighborhood: Physical locations within Ames city limits

### FIXME: This can be easily improved by looking at a map and seeing what neighboorhoods belong with each other

In [16]:
# This can be done A LOT better if I looked at a map!
neighborhood_map = {"NAmes": 0, "CollgCr": 1, "OldTown": 2, "Edwards": 3, "Somerst": 4,
                 "Gilbert": 5, "NridgHt": 6, "Sawyer": 7, "NWAmes": 0, "SawyerW": 7,
                  "BrkSide": 8, "Crawfor": 9, "Mitchel": 10, "NoRidge": 10, "Timber": 10,
                  "IDOTRR": 10, "ClearCr": 10, "SWISU": 10, "StoneBr": 10, "Blmngtn": 10,
                "MeadowV": 10, "BrDale": 10, "Veenker": 10, "NPkVill": 10, "Blueste": 10,
                 }

for dataset in full_data:
    dataset['Neighborhood'] = dataset['Neighborhood'].map(neighborhood_map)

#### Condition1: Proximity to main road or railroad

In [17]:
condition1_map = {"Norm": 0, "Feedr": 1, "Artery": 1, "RRAn": 1, "PosN": 1,
                 "RRAe": 1, "PosA": 1, "RRNn": 1, "RRNe": 1 }

for dataset in full_data:
    dataset['Condition1'] = dataset['Condition1'].map(condition1_map)

#### BldgType: Type of dwelling

In [18]:
bldg_type_map = {"1Fam": 0, "TwnhsE": 1, "Duplex": 2, "Twnhs": 2, "2fmCon": 2}

for dataset in full_data:
    dataset['BldgType'] = dataset['BldgType'].map(bldg_type_map)

#### HouseStyle: Style of dwelling

In [19]:
house_style_map = {"1Story": 0, "2Story": 1, "1.5Fin": 2, "SLvl": 3, "SFoyer": 3,
                "1.5Unf": 3, "2.5Unf": 3, "2.5Fin": 3}

for dataset in full_data:
    dataset['HouseStyle'] = dataset['HouseStyle'].map(house_style_map)

#### Fence: Fence quality

In [20]:
fence_map = {"MnPrv": 1, "GdPrv": 2, "GdWo": 3, "MnWw": 0}

for dataset in full_data:
    dataset['Fence'] = dataset['Fence'].map(fence_map)
    
training_data["Fence"].fillna(0, inplace=True)
testing_data["Fence"].fillna(0, inplace=True)

#### Electrical: Electrical system

In [21]:
electrical_map = {"SBrkr": 1, "FuseA": 2, "FuseF": 0, "FuseP": 0, "Mix": 0}

for dataset in full_data:
    dataset['Electrical'] = dataset['Electrical'].map(electrical_map)

#### CentralAir: Central air conditioning

In [22]:
central_air_map = {"N": 0, "Y": 1}

for dataset in full_data:
    dataset['CentralAir'] = dataset['CentralAir'].map(central_air_map)

#### BsmtFinType1: Quality of basement finished area

In [23]:
bsmt_fin_typ1_map = {"Unf": 0, "LwQ": 1, "Rec": 2, "BLQ": 3, "ALQ": 4, "GLQ": 5}

for dataset in full_data:
    dataset['BsmtFinType1'] = dataset['BsmtFinType1'].map(bsmt_fin_typ1_map)

#### Exterior1st: Exterior covering on house

In [24]:
# If I knew more I could quantify these better.  I'm guessing stone is the most expensive
exterior_1_map = {"VinylSd": 10, "HdBoard": 9, "MetalSd": 8, "Wd Sdng": 7, "Plywood": 6, "CemntBd": 5,
                 "BrkFace": 4, "WdShing": 3, "Stucco": 2, "AsbShng": 1, "BrkComm": 0, "Stone": 0,
                 "CBlock": 0, "ImStucc": 0, "AsphShn": 0}

for dataset in full_data:
    dataset['Exterior1st'] = dataset['Exterior1st'].map(exterior_1_map)

#### Exterior2nd: Exterior covering on house (if more than one material)

In [25]:
# If I knew more I could quantify these better.  I'm guessing stone is the most expensive
exterior_2_map = {"VinylSd": 10, "HdBoard": 9, "MetalSd": 8, "Wd Sdng": 7, "Plywood": 6, "CemntBd": 5,
                 "BrkFace": 4, "WdShing": 3, "Stucco": 2, "AsbShng": 1, "BrkComm": 0, "Stone": 0,
                 "CBlock": 0, "ImStucc": 0, "AsphShn": 0, "Brk Cmn": 0, "Other": 0}

for dataset in full_data:
    dataset['Exterior2nd'] = dataset['Exterior2nd'].map(exterior_2_map)

#### Foundation: Type of foundation

In [26]:
foundation_map = {"PConc": 3, "CBlock": 2, "BrkTil": 1, "Slab": 0, "Stone": 0, "Wood": 0}

for dataset in full_data:
    dataset['Foundation'] = dataset['Foundation'].map(foundation_map)

### Predictions!!

In [27]:
# ignoring street, Utilities, Condition2, RoofMatl, Exterior1st, Exterior2nd, Foundation, MasVnrType, TotalBsmtSF

predictor_cols = ['Quality', '1stFlrSF','2ndFlrSF', 'Alley', 'BedroomAbvGr', 'BldgType', 'BsmtFinType1',
                  'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'CentralAir', 
                  'Condition1', 'Electrical',
                  'Fence', 'Fireplaces', 'FullBath', 'GarageArea',
                 'GarageCars', 'GrLivArea', 'HalfBath', 'HouseStyle', 'KitchenAbvGr',
                  'LandSlope', 'LotArea', 'LotConfig',
                 'LotFrontage', 'LotShape', 'LowQualFinSF', 'MasVnrArea', 'MiscVal', 'MSSubClass',
                  'MSZoning', 'MoSold', 'Neighborhood', 'OpenPorchSF',
                 'OverallCond', 'OverallQual', 'PoolArea', 'SaleCondition', 'SaleType', 'ScreenPorch', 'TotRmsAbvGrd', 
                  'WoodDeckSF', 'YearBuilt', 'YearRemodAdd', 'YrSold']

train_x = training_data[predictor_cols]
train_y = training_data.SalePrice
test_x = testing_data[predictor_cols]

len(predictor_cols)

### Cross Validation scores

Using r2
XGBoost
- 18 predictors = 0.7702056269772473
- 24 predictors = 0.8018358449730526
- 27 predictors = 0.8299348333411789
- 29 predictors = 0.8479059750863349
- 38 predictors = 0.8685503276750433
- 41 predictors = 0.8883098714862785
- 53 predictors = 0.8796081570983425

XGBoost Quality col
- 47 predictors = 0.8406845451878138

SVC
- 27 predictors = -0.27731475311827614

RandomForestRegressor
- 27 predictors = 0.7931188734757507

In [32]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

pipeline = make_pipeline(Imputer(), RandomForestRegressor())

scoring = 'accuracy'
score = cross_val_score(pipeline, train_x, train_y, cv=k_fold, n_jobs=1, scoring="r2")
print(score)
print("\nAverage is ...")
print(sum(score) / len(score))

[0.84095884 0.77875741 0.84975802 0.8448382  0.7620946  0.89233942
 0.8335226  0.86740208 0.86842854 0.86874574]

Average is ...
0.8406845451878138


In [40]:
pipeline.fit(train_x, train_y)
prediction = pipeline.predict(test_x)

submission = pd.DataFrame({
        "Id": testing_data["Id"],
        "SalePrice": prediction
    })

submission.to_csv('/Users/pbezuhov/git/Kaggle/submissions/6_unstable_r2.csv', index=False)